# Neural Network Model

In [885]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#Selecting features based on your analysis
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error


## Data Processing

In [886]:
listStart = 0
listEnd = 0
predictData =  pd.DataFrame
playersList = pd.DataFrame
num_of_features = 10,
setSelectedFeatures = list[str]

In [887]:
master_table_final = pd.read_csv("D:\School Programs\SE4482\master_table_final.csv")
master_table_final_fatigue = pd.read_csv("D:\School Programs\SE4482\master_table_final_fatigue.csv")

#Remove Duplicates:
master_table_final_fatigue.drop_duplicates(inplace=True)
master_table_final.drop_duplicates(inplace=True)

#Handle Missing Values:
#For simplicity, fill missing numeric values with the median and drop rows where categorical data (like Player or Tm) is missing.
numeric_cols = master_table_final_fatigue.select_dtypes(include=['float64', 'int64']).columns
master_table_final_fatigue[numeric_cols] = master_table_final_fatigue[numeric_cols].fillna(master_table_final_fatigue[numeric_cols].median())
master_table_final_fatigue.dropna(subset=['Player', 'Tm'], inplace=True)

#One-Hot Encoding: Convert categorical variables like Tm to a one-hot encoded format.
master_table_final_fatigue = pd.get_dummies(master_table_final_fatigue, columns=['Tm'])

remove_feature = ['W','Player', 'Share', 'WS', 'WS/48', 'Pts Won', 'Rank', 'First', 'usage_percentage', 'win_shares', 'defensive_win_shares' ]


def get_encode_data(X_data):

    X_encoded = pd.get_dummies(X_data, drop_first=True)
    
    # Ensure no missing values
    X_encoded.fillna(X_encoded.median(), inplace=True)

    return X_encoded

def process_data(data, setSelectedFeatures):

    
    data.drop_duplicates(inplace = True)
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())
    if 'Tm' in data.columns:
            data.dropna(subset=['Player', 'Tm'], inplace=True)
            data = pd.get_dummies(data, columns=['Tm'])
    else:
        print("'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.")
            
    X_data = data.drop(remove_feature, axis=1)  # Dropping non-feature columns and WS duplicate
    y_data = data['Share']  # MVP Share is the target

    X_encoded = get_encode_data(X_data)

    X_data = X_encoded[setSelectedFeatures]

    return y_data , X_data



## Feature Selection

In [888]:
def retrieve_Feature_Mutual_Info(X_data, y_data):
    
    # Compute Mutual Information
    mi_scores = mutual_info_regression(X_data, y_data, random_state=42)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_data.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    # print("mi_scores")
    # display(mi_scores)

    return mi_scores

In [889]:
#Feature Selection

selected_features_fatigue = [
    # Basic stats
    'PTS', 'AST', 'TRB',
    #Domain Knowledge stats
    'seed', 'value_over_replacement_player', "box_plus_minus","win_shares_per_48_minutes" ,
    # Consider MVP Fatigue
    'won1yearago', 'won2yearsago'
] 

selected_features = [
    # Basic stats
    'PTS', 'AST', 'TRB',

    #Domain Knowledge stats
    'seed', 'value_over_replacement_player', "box_plus_minus","win_shares_per_48_minutes" ,
] 

def setFeatures(max_number_of_features, fatigue, data):


    setDataSetX = data.drop(remove_feature, axis=1)  # Dropping non-feature columns and WS duplicate
    setDataSety = data['Share']

      #2. Handle Categorical Variables
    #If your dataset contains categorical features, you should encode them as Mutual Information can be computed for continuous variables. For simplicity, you might convert categorical variables to a one-hot encoded format.
    X_encoded = get_encode_data(setDataSetX)
    mi_scores = retrieve_Feature_Mutual_Info(X_encoded, setDataSety)


    
    if(fatigue):

      #Remove duplicates
      feature_list = selected_features_fatigue + mi_scores.index[:max_number_of_features].tolist()
      seen = set()
      unique_list = [x for x in feature_list if not (x in seen or seen.add(x))]
      
      return unique_list
      
    else:

      #Remove duplicates
      feature_list = selected_features + mi_scores.index[:max_number_of_features].tolist()
      seen = set()
      unique_list = [x for x in feature_list if not (x in seen or seen.add(x))]

      return feature_list


## Implementing NN Model

In [890]:
def useMVPYear(year, fatigue):
    
    if(fatigue):

        predictData = master_table_final_fatigue

    else: 
         predictData = master_table_final

    print("selected data:")
    display(predictData)

    setSelectedFeatures = setFeatures(10, fatigue, predictData)

    print("selected_features_fatigue")
    display(setSelectedFeatures )

    #map years to total data List
    start_index, end_index = year_indices[year]

    mvpYearData = predictData[start_index:end_index]

    y_proccesed_data, X_proccessed_data = process_data(mvpYearData,setSelectedFeatures)
    
    playersList = mvpYearData['Player']


    return playersList , y_proccesed_data , X_proccessed_data

year_indices = {
    1980: (0, 8),
    1981: (9, 40),
    1982: (41, 65),
    1983: (66, 88),
    1984: (89, 103),
    1985: (104, 126),
    1986: (127, 144),
    1987: (145, 161),
    1988: (162, 178),
    1989: (179, 197),
    1990: (198, 211),
    1991: (212, 231),
    1992: (232, 248),
    1993: (249, 262),
    1994: (263, 278),
    1995: (279, 292),
    1996: (293, 309),
    1997: (310, 329),
    1998: (330, 348),
    1999: (349, 369),
    2000: (370, 385),
    2001: (386, 402),
    2002: (403, 420),
    2003: (421, 433),
    2004: (434, 449),
    2005: (450, 464),
    2006: (465, 475),
    2007: (476, 492),
    2008: (493, 509),
    2009: (510, 520),
    2010: (521, 534),
    2011: (535, 547),
    2012: (548, 562),
    2013: (563, 578),
    2014: (579, 595),
    2015: (596, 607),
    2016: (608, 617),
    2017: (618, 628),
    2018: (629, 641),
    2019: (642, 653),
    2020: (654, 665),
    2021: (666, 678),
    2022: (679, 690),
    2023: (691, 703),
}

In [891]:
def useAllMVPYears(fatigue):
    
    if(fatigue):

        predictData = master_table_final_fatigue

    else: 
         predictData = master_table_final

    # print("selected data:")
    # display(predictData)

    features_selected = setFeatures(10, fatigue, predictData)

    # print("selected_features_fatigue")
    # display(features_selected)

    y_proccesed_data, X_proccessed_data = process_data(predictData,features_selected)
    
    playersList = predictData['Player']

    return playersList , y_proccesed_data , X_proccessed_data

In [892]:
#SET TEST CONDITIONS
fatigue = True
mvpYear = 0

#train model
players_proccessed_List , y_proccessed_data , X_proccessed_data = useAllMVPYears(fatigue)
X_train, X_test, y_train, y_test, players_list_train, players_list_test = train_test_split(X_proccessed_data, y_proccessed_data, players_proccessed_List, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer: Predicting a single value (MVP Share)
])

# Fitting the model with the training data
model.compile(optimizer='adam',  # Optimizer
              loss='mean_squared_error',  # Loss function to minimize
              metrics=['mean_absolute_error'])  # Additional metrics

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

if mvpYear == 0:
    players_proccessed_List = players_list_test
    y_pred = model.predict(X_test_scaled)

else:

    players_mvp_year_proccessed_List , y_mvp_year_data , X_mvp_year_data = useMVPYear(mvpYear, fatigue)
    players_proccessed_List = players_mvp_year_proccessed_List

    X_mvp_year_test_scaled = scaler.transform(X_mvp_year_data)

    y_pred = model.predict(X_mvp_year_test_scaled)

# print("data:")
# display(X_proccessed_data)
print("shares:")
display(y_proccessed_data)
print("players:")
display(players_proccessed_List)

# Predicting the MVP Share for the test set
predicted_mvp_shares = pd.DataFrame({
    'Player': players_proccessed_List,
    'Predicted MVP Share': y_pred.flatten()
})

top_mvp_candidates = predicted_mvp_shares.sort_values(by='Predicted MVP Share', ascending=False).head(len(predicted_mvp_shares))
print('Predicted for year', mvpYear)
print(top_mvp_candidates)



'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
Epoch 1/50


C:\Users\warre\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.1036 - mean_absolute_error: 0.2277 - val_loss: 0.0473 - val_mean_absolute_error: 0.1675
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0445 - mean_absolute_error: 0.1642 - val_loss: 0.0243 - val_mean_absolute_error: 0.1198
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0291 - mean_absolute_error: 0.1295 - val_loss: 0.0246 - val_mean_absolute_error: 0.1144
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0253 - mean_absolute_error: 0.1179 - val_loss: 0.0227 - val_mean_absolute_error: 0.1086
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0263 - mean_absolute_error: 0.1199 - val_loss: 0.0198 - val_mean_absolute_error: 0.0969
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0220 - mean_absolute_error: 0.1064 - val_loss: 0.0185 - val_mean_absolute_error: 0.0942
Epoch 7/50
 1/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0244 - mean_absolute_error: 0.1028

## Evaluate the Model

In [ ]:
# Calculate the model performance metrics



def eval_model( X_compare, y_compare ,y_pred):

    test_loss, test_mae = model.evaluate(X_compare, y_pred)
    
    #accurate mse
    mse = mean_squared_error(y_compare, y_pred)

    r2 = r2_score(y_compare, y_pred)

    print(f"Num of Features: {num_of_features}")
    print(f"Mean Squared Error: {mse}{test_loss}")
    print(f"Mean Absolute Error: {test_mae}")
    print(f"R-squared: {r2}")

    return mse , r2
if mvpYear == 0:

    eval_model(X_test_scaled, y_pred, y_test)


else:

    eval_model(X_mvp_year_test_scaled, y_pred, y_mvp_year_data )



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0247 - mean_absolute_error: 0.1058 
Num of Features: (10,)
Mean Squared Error: 0.0265617733263743480.02680528722703457
Mean Absolute Error: 0.11209102720022202
R-squared: 0.46976995367580976


In [ ]:
import numpy as np

# Function to simulate training and evaluation of the NN model 10 times
def simulate_nn_training_and_evaluation():
    mse_scores = []
    r2_scores = []
    for _ in range(5):

        fatigue = True
        mvpYear = 0

        #train model
        players_proccessed_List , y_proccessed_data , X_proccessed_data = useAllMVPYears(fatigue)
        X_train, X_test, y_train, y_test, players_list_train, players_list_test = train_test_split(X_proccessed_data, y_proccessed_data, players_proccessed_List, test_size=0.2, random_state=42)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        model = Sequential([
            Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
            Dense(64, activation='relu'),
            Dense(1)  # Output layer: Predicting a single value (MVP Share)
        ])

        # Fitting the model with the training data
        model.compile(optimizer='adam',  # Optimizer
                    loss='mean_squared_error',  # Loss function to minimize
                    metrics=['mean_absolute_error'])  # Additional metrics

        history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

        if mvpYear == 0:
            players_proccessed_List = players_list_test
            y_pred = model.predict(X_test_scaled)

        else:

            players_mvp_year_proccessed_List , y_mvp_year_data , X_mvp_year_data = useMVPYear(mvpYear, fatigue)
            players_proccessed_List = players_mvp_year_proccessed_List

            X_mvp_year_test_scaled = scaler.transform(X_mvp_year_data)

            y_pred = model.predict(X_mvp_year_test_scaled)


        mse, r2 =  eval_model(X_test_scaled, y_pred, y_test)# X_compare is not used in eval_model, thus set to None
        mse_scores.append(mse)
        r2_scores.append(r2)
    # Calculate and print the average MSE and its standard deviation
    average_mse = np.mean(mse_scores)
    std_dev_mse = np.std(mse_scores)
    average_r2 = np.mean(r2_scores)

    return average_mse, std_dev_mse, average_r2

# Calling the simulation function
average_mse, std_dev_mse, average_r2 = simulate_nn_training_and_evaluation()
print(f"Average MSE: {average_mse}, Standard Deviation of MSE: {std_dev_mse}")
print(f"Average R2: {average_r2}")



'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
Epoch 1/50


C:\Users\warre\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1361 - mean_absolute_error: 0.2697 - val_loss: 0.0436 - val_mean_absolute_error: 0.1594
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0463 - mean_absolute_error: 0.1591 - val_loss: 0.0254 - val_mean_absolute_error: 0.1161
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0323 - mean_absolute_error: 0.1275 - val_loss: 0.0217 - val_mean_absolute_error: 0.1038
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0254 - mean_absolute_error: 0.1143 - val_loss: 0.0170 - val_mean_absolute_error: 0.0914
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0254 - mean_absolute_error: 0.1089 - val_loss: 0.0171 - val_mean_absolute_error: 0.0923
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0216 - mean_absolute_error: 0.1009 - val_loss: 0.0173 - val_mean_absolute_error: 0.0949
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0200 - mean_absolute_error: 0.0953 - val_loss: 0.0168 - val_me

C:\Users\warre\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2498 - mean_absolute_error: 0.3422 - val_loss: 0.0558 - val_mean_absolute_error: 0.1849
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0608 - mean_absolute_error: 0.1774 - val_loss: 0.0368 - val_mean_absolute_error: 0.1347
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0411 - mean_absolute_error: 0.1463 - val_loss: 0.0298 - val_mean_absolute_error: 0.1168
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0321 - mean_absolute_error: 0.1262 - val_loss: 0.0240 - val_mean_absolute_error: 0.1096
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0267 - mean_absolute_error: 0.1147 - val_loss: 0.0239 - val_mean_absolute_error: 0.1080
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0296 - mean_absolute_error: 0.1204 - val_loss: 0.0226 - val_mean_absolute_error: 0.1030
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0280 - mean_absolute_error: 0.1175 - val_loss: 0.0227 - val_mea

...

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0981 - mean_absolute_error: 0.2371 - val_loss: 0.0428 - val_mean_absolute_error: 0.1374
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0375 - mean_absolute_error: 0.1454 - val_loss: 0.0309 - val_mean_absolute_error: 0.1202
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0301 - mean_absolute_error: 0.1278 - val_loss: 0.0224 - val_mean_absolute_error: 0.1046
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0240 - mean_absolute_error: 0.1109 - val_loss: 0.0223 - val_mean_absolute_error: 0.1061
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0250 - mean_absolute_error: 0.1117 - val_loss: 0.0225 - val_mean_absolute_error: 0.1058
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0186 - mean_absolute_error: 0.0989 - val_loss: 0.0227 - val_mean_absolute_error: 0.1018
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0213 - mean_absolute_error: 0.1011 - val_loss: 0.0215 - val_mea

C:\Users\warre\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0811 - mean_absolute_error: 0.2104 - val_loss: 0.0307 - val_mean_absolute_error: 0.1301
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0328 - mean_absolute_error: 0.1380 - val_loss: 0.0212 - val_mean_absolute_error: 0.1105
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0307 - mean_absolute_error: 0.1284 - val_loss: 0.0206 - val_mean_absolute_error: 0.1117
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0279 - mean_absolute_error: 0.1212 - val_loss: 0.0178 - val_mean_absolute_error: 0.0979
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0223 - mean_absolute_error: 0.1075 - val_loss: 0.0170 - val_mean_absolute_error: 0.0985
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0227 - mean_absolute_error: 0.1045 - val_loss: 0.0171 - val_mean_absolute_error: 0.0946
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0223 - mean_absolute_error: 0.1007 - val_loss: 0.0191 - val_mea

C:\Users\warre\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1764 - mean_absolute_error: 0.3179 - val_loss: 0.0671 - val_mean_absolute_error: 0.1990
Epoch 2/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0473 - mean_absolute_error: 0.1661 - val_loss: 0.0417 - val_mean_absolute_error: 0.1293
Epoch 3/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0335 - mean_absolute_error: 0.1319 - val_loss: 0.0345 - val_mean_absolute_error: 0.1213
Epoch 4/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0266 - mean_absolute_error: 0.1182 - val_loss: 0.0331 - val_mean_absolute_error: 0.1141
Epoch 5/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0246 - mean_absolute_error: 0.1121 - val_loss: 0.0298 - val_mean_absolute_error: 0.1076
Epoch 6/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0234 - mean_absolute_error: 0.1057 - val_loss: 0.0286 - val_mean_absolute_error: 0.1045
Epoch 7/50
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0203 - mean_absolute_error: 0.0986 - val_loss: 0.0288 - val_mea

In [ ]:
import pandas as pd


def extract_actual_mvps(year_indices, dataset):

    actual_mvps = {}
    for year, (start_index, _) in year_indices.items():
        # Assuming the MVP is the first player listed in the segment for each year
        actual_mvp = dataset.iloc[start_index]['Player']
        actual_mvps[year] = actual_mvp
    return actual_mvps

actualVPS = extract_actual_mvps(year_indices, master_table_final_fatigue)
print(actualVPS)

def count_correct_mvp_predictions(year_indices, fatigue, rf_model, actual_mvps_dict):
    correct_predictions = 0
    
    for year in year_indices.keys():
        # Use the model to predict MVP shares for the given year
        players_mvp_year_processed_list, _, X_mvp_year_data = useMVPYear(year, fatigue)
        y_pred = rf_model.predict(X_mvp_year_data)
        
        # Create a DataFrame of predicted shares
        predicted_mvp_shares = pd.DataFrame({
            'Player': players_mvp_year_processed_list,
            'Predicted MVP Share': y_pred.flatten()
        }).sort_values(by='Predicted MVP Share', ascending=False)
        
        # Assume the player with the highest predicted share is the predicted MVP
        predicted_mvp = predicted_mvp_shares.iloc[0]['Player']
        
        # Get the actual MVP for the year from the actual_mvps_dict dictionary
        actual_mvp = actual_mvps_dict[year]
        
        # Check if the prediction was correct
        if predicted_mvp == actual_mvp:
            correct_predictions += 1
    
    return correct_predictions


correct_predictions = count_correct_mvp_predictions(year_indices,fatigue,model,actualVPS)

print("here",correct_predictions)
print(correct_predictions/len(actualVPS))

{1980: 'Kareem Abdul-Jabbar', 1981: 'Julius Erving', 1982: 'Moses Malone', 1983: 'Moses Malone', 1984: 'Larry Bird', 1985: 'Larry Bird', 1986: 'Larry Bird', 1987: 'Magic Johnson', 1988: 'Michael Jordan', 1989: 'Magic Johnson', 1990: 'Magic Johnson', 1991: 'Michael Jordan', 1992: 'Michael Jordan', 1993: 'Charles Barkley', 1994: 'Hakeem Olajuwon', 1995: 'David Robinson', 1996: 'Michael Jordan', 1997: 'Karl Malone', 1998: 'Michael Jordan', 1999: 'Karl Malone', 2000: "Shaquille O'Neal", 2001: 'Allen Iverson', 2002: 'Tim Duncan', 2003: 'Tim Duncan', 2004: 'Kevin Garnett', 2005: 'Steve Nash', 2006: 'Steve Nash', 2007: 'Dirk Nowitzki', 2008: 'Kobe Bryant', 2009: 'LeBron James', 2010: 'LeBron James', 2011: 'Derrick Rose', 2012: 'LeBron James', 2013: 'LeBron James', 2014: 'Kevin Durant', 2015: 'Stephen Curry', 2016: 'Stephen Curry', 2017: 'Russell Westbrook', 2018: 'James Harden', 2019: 'Giannis Antetokounmpo', 2020: 'Giannis Antetokounmpo', 2021: 'Nikola Jokić', 2022: 'Nikola Jokić', 2023: 'Jo

,Rank,Player,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,...,Tm_PHO,Tm_POR,Tm_SAC,Tm_SAS,Tm_SDC,Tm_SEA,Tm_TOR,Tm_UTA,Tm_WAS,Tm_WSB
0,1,Kareem Abdul-Jabbar,32,147.0,147.0,221,0.665,82,38.3,24.8,...,False,False,False,False,False,False,False,False,False,False
1,2,Julius Erving,29,31.5,31.5,221,0.143,78,36.1,26.9,...,False,False,False,False,False,False,False,False,False,False
2,3,George Gervin,27,19.0,19.0,221,0.086,78,37.6,33.1,...,False,False,False,True,False,False,False,False,False,False
3,4,Larry Bird,23,15.0,15.0,221,0.068,82,36.0,21.3,...,False,False,False,False,False,False,False,False,False,False
4,5,Dan Roundfield,26,2.0,2.0,221,0.009,81,32.0,16.5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,9,Stephen Curry,34,0.0,5.0,1000,0.005,56,34.7,29.4,...,False,False,False,False,False,False,False,False,False,False
700,10,Jimmy Butler,33,0.0,3.0,1000,0.003,64,33.4,22.9,...,False,False,False,False,False,False,False,False,False,False
701,11,De'Aaron Fox,25,0.0,2.0,1000,0.002,73,33.4,25.0,...,False,False,True,False,False,False,False,False,False,False
702,12,Ja Morant,23,0.0,1.0,1000,0.001,61,31.9,26.2,...,False,False,False,False,False,False,False,False,False,False


selected_features_fatigue


['PTS',
 'AST',
 'TRB',
 'seed',
 'value_over_replacement_player',
 'box_plus_minus',
 'win_shares_per_48_minutes',
 'won1yearago',
 'won2yearsago',
 'player_efficiency_rating',
 'offensive_box_plus_minus',
 'offensive_win_shares',
 'W/L%',
 'true_shooting_percentage']

'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
selected data:


...

,Rank,Player,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,...,Tm_PHO,Tm_POR,Tm_SAC,Tm_SAS,Tm_SDC,Tm_SEA,Tm_TOR,Tm_UTA,Tm_WAS,Tm_WSB
0,1,Kareem Abdul-Jabbar,32,147.0,147.0,221,0.665,82,38.3,24.8,...,False,False,False,False,False,False,False,False,False,False
1,2,Julius Erving,29,31.5,31.5,221,0.143,78,36.1,26.9,...,False,False,False,False,False,False,False,False,False,False
2,3,George Gervin,27,19.0,19.0,221,0.086,78,37.6,33.1,...,False,False,False,True,False,False,False,False,False,False
3,4,Larry Bird,23,15.0,15.0,221,0.068,82,36.0,21.3,...,False,False,False,False,False,False,False,False,False,False
4,5,Dan Roundfield,26,2.0,2.0,221,0.009,81,32.0,16.5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,9,Stephen Curry,34,0.0,5.0,1000,0.005,56,34.7,29.4,...,False,False,False,False,False,False,False,False,False,False
700,10,Jimmy Butler,33,0.0,3.0,1000,0.003,64,33.4,22.9,...,False,False,False,False,False,False,False,False,False,False
701,11,De'Aaron Fox,25,0.0,2.0,1000,0.002,73,33.4,25.0,...,False,False,True,False,False,False,False,False,False,False
702,12,Ja Morant,23,0.0,1.0,1000,0.001,61,31.9,26.2,...,False,False,False,False,False,False,False,False,False,False


selected_features_fatigue


['PTS',
 'AST',
 'TRB',
 'seed',
 'value_over_replacement_player',
 'box_plus_minus',
 'win_shares_per_48_minutes',
 'won1yearago',
 'won2yearsago',
 'player_efficiency_rating',
 'offensive_box_plus_minus',
 'offensive_win_shares',
 'W/L%',
 'true_shooting_percentage']

'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
here 20
0.45454545454545453


C:\Users\warre\AppData\Local\Temp\ipykernel_20108\4124209387.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace = True)
C:\Users\warre\AppData\Local\Temp\ipykernel_20108\4124209387.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())
